## Xây dựng Search Engine trên sàn thương mại điện tử Amazon
### 1. Sử dụng kỹ thuật Web Scraping để lấy dữ liệu sản phẩm từ sàn amazon

1. Thêm thư viện [Selenium](https://www.selenium.dev/) và các thư viện cần thiết khác.
2. Định nghĩa các biến `URL` tới trang chủ amazon, `SESSION` phục vụ ghi log

In [97]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager
import time, datetime
import os
# define variables 
SESSION = datetime.datetime.now().strftime("%d%m%Y%H%M")
URL = "https://www.amazon.com/"

3. Tạo file log

In [113]:
# create log
logs_file = open('logs/session{0}.log'.format(SESSION),'w+',encoding='utf-8');

4. Khởi chạy Chrome, truy cập trang chủ amazon, ghi log

In [114]:
# start webdriver Chrome
web = webdriver.Chrome(executable_path=ChromeDriverManager().install())
web.get(url=URL)
logs_file.write("------------------------------------\nStart new session: {0}\n".format(SESSION))

<ipython-input-114-ef406e5bf926>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  web = webdriver.Chrome(executable_path=ChromeDriverManager().install())


69

5. Danh sách các danh mục sản phẩm trên amazon

In [115]:
# get categories
_search_dropdown = Select(web.find_element(By.ID, 'searchDropdownBox'))
categories = _search_dropdown.options

# convert WebElement to String
categories = [category.text for category in categories]
# remove 'All'
categories.remove('All Departments')
logs_file.write("All category: {0}\n".format(', '.join(categories)))
logs_file.write("Total category: {0}\n".format(len(categories)))
print(categories)

['Arts & Crafts', 'Automotive', 'Baby', 'Beauty & Personal Care', 'Books', "Boys' Fashion", 'Computers', 'Deals', 'Digital Music', 'Electronics', "Girls' Fashion", 'Health & Household', 'Home & Kitchen', 'Industrial & Scientific', 'Kindle Store', 'Luggage', "Men's Fashion", 'Movies & TV', 'Music, CDs & Vinyl', 'Pet Supplies', 'Prime Video', 'Software', 'Sports & Outdoors', 'Tools & Home Improvement', 'Toys & Games', 'Video Games', "Women's Fashion"]


6. Định nghĩa các hàm để lưu đường dẫn đến các sản phẩm trên amazon

In [116]:
#get all link of products
def get_products_one_page(page, category):
    try:
        _product = web.find_elements(By.XPATH, "//div[contains(@class,'s-title-instructions-style')]//a")
        products_file = open('data/{0}.txt'.format(category),'a+',encoding='utf-8')
        for product in _product:
            products_file.write(str(product.get_attribute("href"))+'\n')
        products_file.close()
        logs_file.write(" ({0}:{1}) ".format(page,len(_product)))
    except:
        web.delete_all_cookies()
        web.refresh()

def get_all_products(start_page, end_page, category):
    logs_file.write("Number product on page:")
    for i in range(start_page,end_page):
        print("Save products at page {0}!".format(i))
        get_products_one_page(i,category)
        print("Save successful! Redirecting page {0}".format(int(i) + 1))
        web.find_element(By.XPATH, "//div[@role='navigation']//a[contains(@class,'s-pagination-next')]").click()
        time.sleep(6)

7. Lưu tất cả sản phẩm thuộc của mỗi danh mục

In [118]:
# get all products in each category
i = 'Industrial & Scientific'
start_page = 360 # default 1
end_page = 401 # not configure
is_start = True
# True - chuyển trang từ trang home sang trang 1 của category i
if is_start:
    # choose category and find
    _search_dropdown.select_by_visible_text(i)
    logs_file.write("--------------{0}--------------\n".format(i))
    web.find_element(By.ID, "nav-search-submit-button").click()
    # wait 5s to load page
    time.sleep(5)
    # click on "See all results"
    web.find_element(By.XPATH, '//a[span="See all results"]').click()
    # wait 5s to load page
    time.sleep(5)
else:
    get_all_products(start_page, end_page, i)

Save products at page 150!
Save successful! Redirecting page 151
Save products at page 151!
Save successful! Redirecting page 152
Save products at page 152!
Save successful! Redirecting page 153
Save products at page 153!
Save successful! Redirecting page 154
Save products at page 154!
Save successful! Redirecting page 155
Save products at page 155!
Save successful! Redirecting page 156
Save products at page 156!
Save successful! Redirecting page 157
Save products at page 157!
Save successful! Redirecting page 158
Save products at page 158!
Save successful! Redirecting page 159
Save products at page 159!
Save successful! Redirecting page 160
Save products at page 160!
Save successful! Redirecting page 161
Save products at page 161!
Save successful! Redirecting page 162
Save products at page 162!
Save successful! Redirecting page 163
Save products at page 163!
Save successful! Redirecting page 164
Save products at page 164!
Save successful! Redirecting page 165
Save products at page 165

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=113.0.5672.127)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x002E8893+48451]
	(No symbol) [0x0027B8A1]
	(No symbol) [0x00185058]
	(No symbol) [0x0016D073]
	(No symbol) [0x001CDEBB]
	(No symbol) [0x001DBFD3]
	(No symbol) [0x001CA0B6]
	(No symbol) [0x001A7E08]
	(No symbol) [0x001A8F2D]
	GetHandleVerifier [0x00548E3A+2540266]
	GetHandleVerifier [0x00588959+2801161]
	GetHandleVerifier [0x0058295C+2776588]
	GetHandleVerifier [0x00372280+612144]
	(No symbol) [0x00284F6C]
	(No symbol) [0x002811D8]
	(No symbol) [0x002812BB]
	(No symbol) [0x00274857]
	BaseThreadInitThunk [0x758D7D59+25]
	RtlInitializeExceptionChain [0x7737B74B+107]
	RtlClearBits [0x7737B6CF+191]
	(No symbol) [0x00000000]


8. Đóng file log và Chrome

In [119]:
logs_file.close()
web.quit()